In [ ]:
import pandas as pd
import altair as alt

#import the data
url = "https://raw.githubusercontent.com/UIUC-iSchool-DataViz/is445_data/main/building_inventory.csv"
df = pd.read_csv(url)
alt.data_transformers.disable_max_rows()

#Fill the missing data with Unknown
df['County'].fillna('Not available', inplace=True)
df['Rep Full Name'].fillna('Not available', inplace=True)
df['Year Constructed'].fillna(df['Year Constructed'].median(), inplace=True)
df['Senator Full Name'].fillna('Unknown', inplace=True)
df['Usage Description 2'].fillna(df['Usage Description 2'].mode()[0], inplace=True)
df['Usage Description 3'].fillna(df['Usage Description 3'].mode()[0], inplace=True)
df['Address'].fillna('Not available', inplace=True)
df['Congressional Full Name'].fillna('Unknown', inplace=True)
df

In [ ]:
df.columns

In [ ]:
df_county = df.groupby(['County', 'Bldg Status']).size().reset_index(name='Count')

In [ ]:
radio_button = alt.binding_radio(options=df['Bldg Status'].unique().tolist(), name='Building Status: ')

select_status = alt.selection_single(fields=['Bldg Status'], bind=radio_button)

bar_chart = alt.Chart(df_county).mark_bar().encode(
    x=alt.X('County:O', sort='-y', title='County'),
    y=alt.Y('Count:Q', title='Number of Buildings'),
    color=alt.Color('Bldg Status:N', title='Building Status'),
    tooltip=['County', 'Bldg Status', 'Count']
).add_selection(
    select_status
).transform_filter(
    select_status
).properties(
    title="Number of Buildings by County",
    width=900,
    height=400
)
bar_chart


In [ ]:
data_filtered = df[['County', 'Usage Description']].dropna()

# Dropdown for County selection
dropdown = alt.binding_select(options=data_filtered['County'].unique().tolist(), name='Select County: ')
county_param = alt.param(name='county', bind=dropdown, value=data_filtered['County'].unique()[0])

stacked_bar = (
    alt.Chart(data_filtered)
    .mark_bar()
    .encode(
        x=alt.X('count():Q', title='Building Count'),
        y=alt.Y('Usage Description:N', title='Usage Description'),
        color=alt.Color('Usage Description:N', title='Usage Description'),
        tooltip=[
            alt.Tooltip('count():Q', title='Building Count'),
            alt.Tooltip('Usage Description:N', title='Usage Description'),
            alt.Tooltip('County:N', title='County'),
        ]
    )
    .add_params(county_param)
    .transform_filter(f"datum.County == {county_param.name}")
    .properties(
        title='Building Count by Usage Description for Each County',
        width=800,
        height=400
    )
)

stacked_bar

In [ ]:
bar_chart.save(r'C:\Users\riyan\homework6.1\assets\visualizations\chart1.json')

stacked_bar.save(r'C:\Users\riyan\homework6.1\assets\visualizations\chart2.json')